**Funding Rate Sequence**

***0. Sum of input events*** 

In [1]:
@input("kwenta").
@bind("kwenta","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Input","events.csv").
@timeMapping("kwenta",0, 0,#T,#T).
@mapping("kwenta", 0, "a", "date").
@mapping("kwenta", 1, "b", "int"). 
@mapping("kwenta", 2, "c", "double").

event(Unix, Input) :- kwenta(Unix,Input).

    % Cumulative sum of input events
b_msum(X,Z):- event(X, Y), Z = msum(Y).
b_count(X,Z,C) :- b_msum(X,Z), C = m:count(X).
b_max(X,K) :- b_count(X,Z,C), K = max(C).
events(Input,Unix) :- b_count(Unix,Input,C), b_max(Unix,C2), C = C2.

 % to save data
b(A, Unix, Input) :- events(Input,Unix)@temporalAtom(A,B,C,D).

@output("b").
@bind("b","csv header=True","/vadalog/disk/notebooks/Data/Kwenta/Output","events_sum.csv").

@timeGranularity("seconds").
@temporal(2022-10-07 20:03:11,2022-10-07 21:58:49).
@library("m:", "monotonic").
@relaxedSafety.

***1. Skew Updates***

In [2]:
@input("kwenta").
@bind("kwenta","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Output","events_sum.csv").
@timeMapping("kwenta",0, 0,#T,#T).
@mapping("kwenta", 0, "a", "date").
@mapping("kwenta", 1, "b", "int").
@mapping("kwenta", 2, "c", "double").

marketSkew(1302.8841612646906,1665165791)@[2022-10-07 20:03:11,2022-10-07 20:03:11].

    % Initialization - load from csv
skew(S) :- marketSkew(S, Unix).
event(Input, Unix) :- kwenta(Unix,Input).

 %-----------------------------
 % Rules
 %-----------------------------
% Evolution of the skew
skew(S) :- [-][1,1] skew(S), not event(_,_).
skew(S) :- [-][1,1] skew(OldS), event(Input, Unix), S = OldS + Input.

% Only select interesting timepoints
skewUpdates(S, Unix) :- [-][1,1] skew(S), event(Input, Unix).

 
    % Export
skewExport(A, Unix, S) :- skewUpdates(S, Unix)@temporalAtom(A,B,C,D).

@output("skewExport").
@bind("skewExport","csv header=True","/vadalog/disk/notebooks/Data/Kwenta/Output","skewUpdates.csv").

@timeGranularity("seconds").
@temporal(2022-10-07 20:03:11,2022-10-07 21:58:49).

***2. Computation of unrecorded funding at events***

In [4]:
@input("kwenta").
@bind("kwenta","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Output","events_sum.csv").
@timeMapping("kwenta",0, 0,#T,#T).
@mapping("kwenta", 0, "a", "date").
@mapping("kwenta", 1, "b", "int").
@mapping("kwenta", 2, "c", "double").

@input("marketSkew").
@bind("marketSkew","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Output","skewUpdates.csv").
@timeMapping("marketSkew",0, 0,#T,#T).
@mapping("marketSkew", 0, "a", "date").
@mapping("marketSkew", 1, "b", "int").
@mapping("marketSkew", 2, "c", "double").

@input("prices").
@bind("prices","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Input","prices.csv").
@timeMapping("prices",0, 0,#T,#T).
@mapping("prices", 0, "a", "date").
@mapping("prices", 1, "b", "double").
@mapping("prices", 2, "c", "int").

time(1665165791)@[2022-10-07 20:03:11,2022-10-07 20:03:11].

    % Initialization
price(P) :- prices(P, Unix).
skew(S) :- marketSkew(Unix, S).
event(Input, Unix) :- kwenta(Unix,Input).

    % Funding rate mechanism

tdiff(T1, T2) :- time(T), T1 = T, T2 = T.
% Time difference since last update of funding rate
tdiff(T1, T2) :- [-][1,1] tdiff(T1, T2), not event(_,_).
tdiff(T1, T2) :- [-][1,1] tdiff(OldT1, OldT), event(Input, Unix), T1 = OldT, T2 = Unix.
diff(T) :- tdiff(T1, T2), event(Input, Unix), T = T2-T1.

% Istantaneous Funding Rate computation
rate(I) :- skew(K), price(P), I = ((-K)/(300000000.0/P)).

clampR(I) :- rate(CI), CI > 1.0, I = 1.0.
clampR(I) :- rate(CI), CI < -1.0, I = -1.0.
clampR(I) :- rate(I), I < 1.0, I > -1.0.

funding(UF) :- clampR(I), price(P), diff(T), UF = I*P*T*0.1/86400.0 .

 % to save data
unrecordedFunding(UF,A,B) :- funding(UF)@temporalAtom(A,B,C,D).

@output("unrecordedFunding").
@bind("unrecordedFunding","csv header=True","/vadalog/disk/notebooks/Data/Kwenta/Output","unrecordedFunding.csv").

@timeGranularity("seconds").
@temporal(2022-10-07 20:03:11,2022-10-07 21:58:49).

***3. Computation of Funding Rate Sequence***

In [5]:
@input("funding").
@bind("funding","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Output","unrecordedFunding.csv").
@timeMapping("funding",1, 2,#T,#T).
@mapping("funding", 0, "a", "double").
@mapping("funding", 1, "b", "date").
@mapping("funding", 2, "c", "date").

fundRateSequenceInit(-190.90766893911191)@[2022-10-07 20:03:11,2022-10-07 20:03:11].
frs(F) :- fundRateSequenceInit(F).
unrecordedFunding(UF) :- funding(UF).

    % Funding Sequence Update
frs(F) :- [-][1,1] frs(F), not unrecordedFunding(_).
frs(F) :- [-][1,1] frs(OldF), unrecordedFunding(UF), F = OldF+UF.
fundingSequence(F) :- frs(F), unrecordedFunding(UF).

 % to save data
b(X,A,B) :- fundingSequence(X)@temporalAtom(A,B,C,D).

@output("b").
@bind("b","csv header=True","/vadalog/disk/notebooks/Data/Kwenta/Output","RatesSequence.csv").
@timeGranularity("seconds").
@temporal(2022-10-07 20:03:11,2022-10-07 21:58:49).

**Margin, Order book and PNL**

In [7]:
@input("transferMargin").
@bind("transferMargin","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Input","transferMargin.csv").
@timeMapping("transferMargin",0, 0,#T,#T).
@mapping("transferMargin", 0, "","date").
@mapping("transferMargin", 1, "", "string").
@mapping("transferMargin", 2, "", "double").
@mapping("transferMargin", 3, "", "int").

@input("modifyPos").
@bind("modifyPos","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Input","modifyPos.csv").
@timeMapping("modifyPos",0, 0,#T,#T).
@mapping("modifyPos", 0, "","date").
@mapping("modifyPos", 1, "", "string").
@mapping("modifyPos", 2, "", "double").
@mapping("modifyPos", 3, "", "int").

@input("closePos").
@bind("closePos","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Input","closePos.csv").
@timeMapping("closePos",0, 0,#T,#T).
@mapping("closePos", 0, "", "date").
@mapping("closePos", 1, "", "string").
@mapping("closePos", 2, "", "int").

@input("withdraw").
@bind("withdraw","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Input","withdraw.csv").
@timeMapping("withdraw",0, 0,#T,#T).
@mapping("withdraw", 0, "","date").
@mapping("withdraw", 1, "", "string").
@mapping("withdraw", 2, "", "int").

@input("prices").
@bind("prices","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Input","prices.csv").
@timeMapping("prices",0, 0,#T,#T).
@mapping("prices", 0, "","date").
@mapping("prices", 1, "", "double").
@mapping("prices", 2, "", "int").

 % Retrieve ETH Futures prices time series
eth_perp(Price) :- prices(Price, Unix).

 %-------------------------------------------------------------------------------
 % Margin management
 %-------------------------------------------------------------------------------
 % When money is transferred the margin is opened
openMargin(Account) :- transferMargin(Account, M).    % Boolean that indicates if the margin is open
openMargin(Account) :-  [-][1,1] openMargin(Account), not withdraw(Account). % This predicates is true until the user withdraw everything

 % The first time money is transferred, margin and position are initialized. 
 % The IntervalOpen predicate is used to activate the rule only at the first transferMargin event
intervalOpen(Account) :- [-][1,1] openMargin(Account).
margin(Account, Margin), position(Account, Position, Notional), fee(Account, Fee) :- not intervalOpen(Account), transferMargin(Account, Margin),
                                                                                     eth_perp(Price), Position = 0.00, Notional = 0.00, Fee = 0.00 .

 % Later deposits (transfer margin events) are managed by the rule below
margin(Account, Margin) :- intervalOpen(Account), transferMargin(Account, Margin).

 % The margin is propagated until the user withdraw everything, accounting also for margin changes (transfers or settlements)
withdrawOrChange(Account) :- withdraw(Account).
withdrawOrChange(Account) :- transferMargin(Account, M).
withdrawOrChange(Account) :- closePos(Account).
margin(Account, Margin) :- [-][1,1] margin(Account, Margin), not withdrawOrChange(Account).

 %-------------------------------------------------------------------------------
 % Orderbook and track position
 %-------------------------------------------------------------------------------
order(Account, NewPosition) :- modifyPos(Account, NewPosition).
order(Account, NewPosition) :- closePos(Account), NewPosition = 0.00 .

 % If no new order is sent, the position remains the same
keepPositionAlive(Account) :- not order(Account, _), openMargin(Account).
position(Account, Position, Notional) :- [-][1,1] position(Account, Position, Notional), keepPositionAlive(Account).

 % Open a new position or modifying an existing one
isAlreadyOpen(Account, Position,Notional) :- [-][1,1] position(Account, Position, Notional).
position(Account, Position, Notional) :-  modifyPos(Account, NewPosition), isAlreadyOpen(Account, OldP,OldN), eth_perp(Price), 
    Position = NewPosition+OldP, Notional = (NewPosition * Price)+OldN.
 % Close position
position(Account, Position, Notional) :- closePos(Account), Position = 0.00, Notional = 0.00.

 %-------------------------------------------------------------------------------
 % Future contract internals - compute PNL
 %-------------------------------------------------------------------------------
settlement(Account,Price) :- closePos(Account), eth_perp(Price).
lastPosition(Account, Position, Notional, Price) :- [-][1,1] position(Account, Position, Notional), settlement(Account,Price).
transfer(Account, PL) :- lastPosition(Account, Position, Notional, Price), PL = (Price * Position - Notional).


 % to save data
pnl(A, Account, PL) :- transfer(Account, PL)@temporalAtom(A,B,C,D).
posEv(A, Account, Position, Notional) :- position(Account, Position, Notional)@temporalAtom(A,B,C,D).


@output("posEv").
@output("pnl").

@bind("pnl","csv header=True","/vadalog/disk/notebooks/Data/Kwenta/Output","pnl.csv").
@bind("posEv","csv header=True","/vadalog/disk/notebooks/Data/Kwenta/Output","position.csv").

@timeGranularity("seconds").
@temporal(2022-10-07 20:03:11,2022-10-07 21:58:49).
@library("math:","math").

**Fees**

In [8]:
@input("marketSkew").
@bind("marketSkew","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Output","skewUpdates.csv").
@timeMapping("marketSkew",0, 0,#T,#T).
@mapping("marketSkew", 0, "a", "date").
@mapping("marketSkew", 1, "b", "int").
@mapping("marketSkew", 2, "c", "double").

@input("prices").
@bind("prices","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Input","prices.csv").
@timeMapping("prices",0, 0,#T,#T).
@mapping("prices", 0, "a", "date").
@mapping("prices", 1, "b", "double").
@mapping("prices", 2, "c", "int").

@input("transferMargin").
@bind("transferMargin","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Input","transferMargin.csv").
@timeMapping("transferMargin",0, 0,#T,#T).
@mapping("transferMargin", 0, "","date").
@mapping("transferMargin", 1, "", "string").
@mapping("transferMargin", 2, "", "double").
@mapping("transferMargin", 3, "", "int").

@input("withdraw").
@bind("withdraw","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Input","withdraw.csv").
@timeMapping("withdraw",0, 0,#T,#T).
@mapping("withdraw", 0, "","date").
@mapping("withdraw", 1, "", "string").
@mapping("withdraw", 2, "", "int").

@input("modifyPos").
@bind("modifyPos","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Input","modifyPos.csv").
@timeMapping("modifyPos",0, 0,#T,#T).
@mapping("modifyPos", 0, "","date").
@mapping("modifyPos", 1, "", "string").
@mapping("modifyPos", 2, "", "double").
@mapping("modifyPos", 3, "", "int").

@input("posEv").
@bind("posEv","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Output","position.csv").
@timeMapping("posEv",0, 0,#T,#T).
@mapping("posEv", 0, "", "date").
@mapping("posEv", 1, "", "string").
@mapping("posEv", 2, "", "double").
@mapping("posEv", 3, "", "double").

@input("closePos").
@bind("closePos","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Input","closePos.csv").
@timeMapping("closePos",0, 0,#T,#T).
@mapping("closePos", 0, "", "date").
@mapping("closePos", 1, "", "string").
@mapping("closePos", 2, "", "int").

 %--------------------------------------------------
 % Elements from the previous chunck needed also here
 %--------------------------------------------------
eth_perp(Price) :- prices(Price, Unix).
position(Account, Size, Notional) :- posEv(Account, Size, Notional).
skew(S) :- marketSkew(Unix, S).

 % When money is transferred the margin is opened
openMargin(Account) :- transferMargin(Account, M).    % Boolean that indicates if the margin is open
openMargin(Account) :-  [-][1,1] openMargin(Account), not withdraw(Account). % This predicates is true until the user withdraw everything

 % The first time money is transferred, margin and position are initialized. 
 % The IntervalOpen predicate is used to activate the rule only at the first transferMargin event
intervalOpen(Account) :- [-][1,1] openMargin(Account).
fee(Account, Fee) :- not intervalOpen(Account), transferMargin(Account, Margin), eth_perp(Price), Fee = 0.00 .

 %-------------------------------------------------------------------------------
 % Future contract internals - compute fees
 %-------------------------------------------------------------------------------
order(Account, NewPosition) :- modifyPos(Account, NewPosition).
order(Account, NewPosition) :- closePos(Account), NewPosition = 0.00 .

keepPositionAlive(Account) :- not order(Account, _), position(Account, Size, Notional).
fee(Account, Fee) :- [-][1,1] fee(Account, Fee), keepPositionAlive(Account).

    % In case of modification of the size
hasfees(Account, Fees) :- [-][1,1] fee(Account, Fees).
fee(Account, Fee) :- modifyPos(Account, Position), hasfees(Account, OldFee), eth_perp(P), skew(S), S>0.00, Position > 0.00, Fee = OldFee + math:abs(Position*0.0035*P).
fee(Account, Fee) :- modifyPos(Account, Position), hasfees(Account, OldFee), eth_perp(P), skew(S), S<0.00, Position > 0.00, Fee = OldFee + math:abs(Position*0.003*P).
fee(Account, Fee) :- modifyPos(Account, Position), hasfees(Account, OldFee), eth_perp(P), skew(S), S>0.00, Position < 0.00, Fee = OldFee + math:abs(Position*0.003*P).
fee(Account, Fee) :- modifyPos(Account, Position), hasfees(Account, OldFee), eth_perp(P), skew(S), S<0.00, Position < 0.00, Fee = OldFee + math:abs(Position*0.0035*P).

    % At closing of the position
closing(Account, Position) :-  closePos(Account), [-][1,1] position(Account, Position, Notional).
finalFees(Account, Fee) :- closing(Account, Position), hasfees(Account, OldFee), eth_perp(P), skew(S), S>0.00, Position < 0.00, Fee = OldFee + math:abs(Position * P * 0.0035).
finalFees(Account, Fee) :- closing(Account, Position), hasfees(Account, OldFee), eth_perp(P), skew(S), S<0.00, Position < 0.00, Fee = OldFee + math:abs(Position * P * 0.003).
finalFees(Account, Fee) :- closing(Account, Position), hasfees(Account, OldFee), eth_perp(P), skew(S), S>0.00, Position > 0.00, Fee = OldFee + math:abs(Position * P * 0.003).
finalFees(Account, Fee) :- closing(Account, Position), hasfees(Account, OldFee), eth_perp(P), skew(S), S<0.00, Position > 0.00, Fee = OldFee + math:abs(Position * P * 0.0035).
fee(Account, Fee) :- closing(Account, Position), Fee = 0.0.

fees(A, Account, Fee) :- finalFees(Account, Fee)@temporalAtom(A,B,C,D).

@output("fees").

@bind("fees","csv header=True","/vadalog/disk/notebooks/Data/Kwenta/Output","fees.csv").

@timeGranularity("seconds").
@temporal(2022-10-07 20:03:11,2022-10-07 21:58:49).
@library("math:","math").

**Individual Accrued Funding**

In [10]:
@input("rates").
@bind("rates","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Output","RatesSequence.csv").
@timeMapping("rates",1, 2,#T,#T).
@mapping("rates", 0, "a", "double").
@mapping("rates", 1, "b", "date").
@mapping("rates", 2, "c", "date").

@input("posEv").
@bind("posEv","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Output","position.csv").
@timeMapping("posEv",0, 0,#T,#T).
@mapping("posEv", 0, "", "date").
@mapping("posEv", 1, "", "string").
@mapping("posEv", 2, "", "double").
@mapping("posEv", 3, "", "double").

@input("modifyPos").
@bind("modifyPos","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Input","modifyPos.csv").
@timeMapping("modifyPos",0, 0,#T,#T).
@mapping("modifyPos", 0, "","date").
@mapping("modifyPos", 1, "", "string").
@mapping("modifyPos", 2, "", "double").
@mapping("modifyPos", 3, "", "int").

@input("closePos").
@bind("closePos","csv useHeaders=true, delimiter=',',nullString='', quoteMode='MINIMAL', 
            recordSeparator='\n'","/vadalog/disk/notebooks/Data/Kwenta/Input","closePos.csv").
@timeMapping("closePos",0, 0,#T,#T).
@mapping("closePos", 0, "", "date").
@mapping("closePos", 1, "", "string").
@mapping("closePos", 2, "", "int").

position(Account, Size, Notional) :- posEv(Account, Size, Notional).
fundRate(R) :- rates(R).

order(Account, NewPosition) :- modifyPos(Account, NewPosition).
order(Account, NewPosition) :- closePos(Account), NewPosition = 0.00 .

lastPos(Account, Size, Notional) :- [-][1,1] position(Account, Size, Notional).
 % Record R when a position is opened
individualF(Account, Position, R, CF) :- modifyPos(Account, Position),fundRate(R),lastPos(Account, Size, Notional), Size = 0.00 , CF = 0.00 .
 % in case of no order this predicate is propagated over time
aux(Account, Position, R, CF) :- [-][1,1] individualF(Account, Position, R, CF).
individualF(Account, Position, R, CF) :- aux(Account, Position, R, CF), not order(Account, _).

 % in case of modify position
individualF(Account, Position, R, CF) :- aux(Account, OldPosition, OldR, OldCF), modifyPos(Account, Size), fundRate(R), 
                                        Position = OldPosition+Size, CF = (OldCF + OldPosition*(R-OldR)). 
 % in case of close position
accruedFunding(Account, F) :- aux(Account, Position, R, CF), closePos(Account),fundRate(AR), F = CF + Position*(AR-R).

 % to save data
b(Account, F, A) :- accruedFunding(Account, F)@temporalAtom(A,B,C,D).

@output("b").
@bind("b","csv header=True","/vadalog/disk/notebooks/Data/Kwenta/Output","individualFund.csv").

@timeGranularity("seconds").
@temporal(2022-10-07 20:03:11,2022-10-07 21:58:49).